In [ ]:
# using SpecialFunctions

In [16]:
function erfc_approx(z)
    e = 2/sqrt(pi)*(z-z^3/3+z^5/10-z^7/42+z^9/216)
#     a = -(8/(3*pi))*(pi-3)/(pi-4)
#     e = sqrt(1-exp(-z^2*(4/pi+a*z^2)/(1+a*z^2)))
#     if (z<0.0) e = -e end
    return 1-e
end

erfc_approx (generic function with 1 method)

In [17]:
function cdf_normal(x)
#     c = 0.5*erfc(-x/sqrt(2))
    c_approx = 0.5*erfc_approx(-x/sqrt(2))
    return c_approx
end

cdf_normal (generic function with 1 method)

In [18]:
cdf_normal(0.1)
# a = -(8/(3*pi))*(pi-3)/(pi-4)


0.5398278372770291

In [21]:
function tauchen(N,mu,rho,sigma,m)

    Z     = zeros(N,1) # グリッド
    Zprob = zeros(N,N) # 遷移確率の行列
    c     = (1-rho)*mu # 定数項

    # 等間隔のグリッドを定める
    # 最大値と最小値
    zmax  = m*sqrt(sigma^2/(1-rho^2))
    zmin  = -zmax
    # グリッド間の間隔
    w = (zmax-zmin)/(N-1)

    Z = collect(LinRange(zmin,zmax,N))
    # Z = linspace(zmin,zmax,N)'
    # 定常状態はmu
    Z = Z .+ mu

    # グリッドを所与として、遷移確率を求める
    for j = 1:N
        for k = 1:N
            if k == 1
                Zprob[j,k] = cdf_normal((Z[1]-c-rho*Z[j]+w/2)/sigma)
            elseif k == N
                Zprob[j,k] = 1 - cdf_normal((Z[N]-c-rho*Z[j]-w/2)/sigma)
            else
                Zprob[j,k] = cdf_normal((Z[k]-c-rho*Z[j]+w/2)/sigma) - cdf_normal((Z[k]-c-rho*Z[j]-w/2)/sigma)
            end
        end
    end
    
    return Z,Zprob
    
end

tauchen (generic function with 1 method)

In [ ]:
struct Model{TI<:Integer, TF<:AbstractFloat, TV<:Vector}
    rstar::TF       # 割引因子
    bet::TF       # 相対的危険回避度
    sig::TF       # 資本分配率
    alp::TF       # 固定資本減耗      
    the::TF
    ome::TF
    kap::TF
    lam::TF
    Ng::TI      # 資本グリッドの個数
    rhog::TF    # 資本グリッドの最大値
    sigg::TF    # 資本グリッドの最小値 (0にすると生産が出来なくなる)
    maxiter::TI # 繰り返し計算の最大値
    tol::TF     # 許容誤差
end

In [ ]:
m.rstar = 3.5/4; % pH=0のときの、定常状態での名目金利の値
m.bet = 1/(1+m.rstar/100); % 割引率(オイラー方程式の定常状態より)
m.sig = 6.25;
m.alp = 0.66;
m.the = 7.66;
m.ome = 0.47;
m.kap = (1-m.alp)*(1-m.alp*m.bet)/m.alp*(1/m.sig+m.ome)/(1+m.ome*m.the); %0.024;
m.lam = 0.048/16; %0.003;

% joint shock process
m.rhou = 0;
m.rhog = 0.8;
m.sigu = 0.154;
m.sigg = 1.524;
m.Ng = 31;
m.Nu = 31;

m.maxiter = 2000; % 繰り返し回数の最大値
m.tol = 1e-5; % 許容誤差